In [ ]:
!! pip install -U open-metric-learning

In [ ]:
# !pip install facenet-pytorch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os
# from PIL import Image
# from facenet_pytorch import MTCNN

# # MTCNN 초기화
# mtcnn = MTCNN(image_size=224, margin=20, min_face_size=20)

# # 파일 경로 지정
# dataset_root = "OML/img/pre_WOMAN"
# cropped_images_dir = "OML/img/WOMAN"

# # 이미지를 동일한 크기로 조정하고 얼굴을 자르는 변환 정의
# def transform_image_with_crop(image_path):
#     image = Image.open(image_path).convert('RGB')
#     boxes, _ = mtcnn.detect(image)
#     if boxes is not None:
#         # 얼굴이 감지된 경우
#         for i, box in enumerate(boxes):
#             x_min, y_min, x_max, y_max = box
#             cropped_face = image.crop((x_min, y_min, x_max, y_max))
#             cropped_face.save(os.path.join(cropped_images_dir, os.path.basename(image_path)))

# # 모든 이미지 파일에 대해 얼굴을 자르고 저장
# for filename in os.listdir(dataset_root):
#     if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
#         transform_image_with_crop(os.path.join(dataset_root, filename))

In [ ]:
import torch
from tqdm import tqdm

from oml.datasets.base import DatasetWithLabels
from oml.losses.triplet import TripletLossWithMiner
from oml.miners.inbatch_all_tri import AllTripletsMiner
from oml.models import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.utils.download_mock_dataset import download_mock_dataset

In [ ]:
import os
os.chdir("OML/img/WOMAN")

# path
fpath = os.listdir()
path = sorted(fpath)

# label
n = 0
label = []
for i in path:
  fst = [word[0] for word in path]
  for letter in fst:
    if letter == fst[0]:
      label.append(n)
    else:
      break
  n += 1

  if len(label) >= len(path):
    break

In [ ]:
# split
split = ["train" for _ in label]
idx = int(len(label) * 0.5)

while(True):
  if label[idx] == label[idx - 1] and label[idx] == label[idx + 1]:
    idx += 1

  elif label[idx] == label[idx - 1]:
    idx += 1

  else: break


for i in range(idx,len(split)):
  split[i] = "validation"

In [ ]:
# is_query, is_gallery
import numpy as np
is_query = ['NaN' for _ in label]
is_gallery = ['NaN' for _ in label]

for i in range(idx,len(label)):
  if label[i] != label[i - 1] and label[i] == label[i + 1]:
    is_query[i] = True
    is_gallery[i] = False
  else:
    is_query[i] = False
    is_gallery[i] = True

In [ ]:
# 테이블 생성
import os
os.chdir("OML/")
import sqlite3

conn = sqlite3.connect('img/w_img.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE w_img
                  (label, path, split, is_query, is_gallery)''')

In [ ]:
import os
import sqlite3
os.chdir("OML/")
conn = sqlite3.connect('img/w_img.db')
cursor = conn.cursor()

In [ ]:
# db에 데이터 삽입
for i in range(len(label)):
    cursor.execute("insert into w_img values (?, ?, ?, ?, ?)", (label[i], path[i], split[i], is_query[i], is_gallery[i]))

conn.commit()

In [ ]:
# DF 타입 출력
import pandas as pd

cursor.execute("SELECT * FROM w_img;")
rows = cursor.fetchall()

cols = [column[0] for column in cursor.description]

df = pd.DataFrame.from_records(data=rows, columns=cols)
conn.close()
df

,label,path,split,is_query,is_gallery
0,0,AnYujin01.jpg,train,NaN,NaN
1,0,AnYujin02.jpg,train,NaN,NaN
2,0,AnYujin03.jpg,train,NaN,NaN
3,1,GoYoonJung01.jpg,train,NaN,NaN
4,1,GoYoonJung02.jpg,train,NaN,NaN
5,1,GoYoonJung03.jpg,train,NaN,NaN
6,2,HanSohee01.jpg,train,NaN,NaN
7,2,HanSohee02.jpg,train,NaN,NaN
8,2,HanSohee03.jpg,train,NaN,NaN
9,3,HeoYoonjin01.jpg,train,NaN,NaN


In [ ]:
import pandas as pd

df_train = df[df['split'] == "train"]
dataset_root = "img/WOMAN/"

In [ ]:
from torchvision import transforms

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 원하는 크기로 조정
    transforms.ToTensor(),  # 텐서로 변환
])

In [ ]:
image_paths = dataset_root + df["path"]

In [ ]:
# import cv2

# for img_path in image_paths:
#     image = cv2.imread(img_path)
#     print(f"이미지 경로: {img_path}, 이미지 크기: {image.shape}")

In [ ]:
model = ViTExtractor("vits16_dino", arch="vits16", normalise_features=False).train()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
train_dataset = DatasetWithLabels(df_train, dataset_root=dataset_root, transform=image_transform)
criterion = TripletLossWithMiner(margin=0.1, miner=AllTripletsMiner(), need_logs=True)

https://dl.fbaipublicfiles.com/dino/dino_deitsmall16_pretrain/dino_deitsmall16_pretrain.pth
Checkpoint is already here.


In [ ]:
sampler = BalanceSampler(train_dataset.get_labels(), n_labels=5, n_instances=20)
# torch.utils.data.DataLoader(): 데이터를 미니배치 형태로 만듦
train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=sampler)

In [ ]:
for batch in tqdm(train_loader):
    embeddings = model(batch["input_tensors"])

    loss = criterion(embeddings, batch["labels"])
    # print("\n",batch["input_tensors"], "\n", batch["labels"])
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # info for logging: positive/negative distances, number of active triplets
    print(criterion.last_logs)

100%|██████████| 1/1 [01:08<00:00, 68.26s/it]

{'active_tri': 0.2719605267047882, 'pos_dist': 36.70737838745117, 'neg_dist': 57.582401275634766}
